📝 **Author:** Amirhossein Heydari - 📧 **Email:** amirhosseinheydari78@gmail.com - 📍 **Linktree:** [linktr.ee/mr_pylin](https://linktr.ee/mr_pylin)

---

# Dependencies

In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2

# Normalization

## 1. Base Normalization
   - Min-Max normalization
   - Z-score normalization

In [2]:
trainset = CIFAR10('./dataset', train=True, transform=None)

# log
print(f"trainset.data.shape : {trainset.data.shape}")
print(f"trainset.data.dtype : {trainset.data.dtype}")
print(f"type(trainset.data) : {type(trainset.data)}")

trainset.data.shape : (50000, 32, 32, 3)
trainset.data.dtype : uint8
type(trainset.data) : <class 'numpy.ndarray'>


### 1.1. Min-Max
   - there is no any built-in feature for this type of normalization in pytorch

In [3]:
min_value = trainset.data.min(axis=(0, 1, 2))
max_value = trainset.data.max(axis=(0, 1, 2))

# log
print(f"Minimum values per channel : {min_value}")
print(f"Maximum values per channel : {max_value}")

Minimum values per channel : [0 0 0]
Maximum values per channel : [255 255 255]


In [4]:
# normalize to the range: (0, 1)
minmax_trainset_1 = (trainset.data - min_value) / (max_value - min_value)

# normalize to the range: (-1, 1)
minmax_trainset_2 = minmax_trainset_1 * 2 - 1

# log
print(f"Minimum values for minmax_trainset_1 : {minmax_trainset_1.min(axis=(0, 1, 2))}")
print(f"Maximum values for minmax_trainset_1 : {minmax_trainset_1.max(axis=(0, 1, 2))}")
print('-' * 50)
print(f"Minimum values for minmax_trainset_2 : {minmax_trainset_2.min(axis=(0, 1, 2))}")
print(f"Maximum values for minmax_trainset_2 : {minmax_trainset_2.max(axis=(0, 1, 2))}")

Minimum values for minmax_trainset_1 : [0. 0. 0.]
Maximum values for minmax_trainset_1 : [1. 1. 1.]
--------------------------------------------------
Minimum values for minmax_trainset_2 : [-1. -1. -1.]
Maximum values for minmax_trainset_2 : [1. 1. 1.]


### 1.2. Z-score
   - there is no any built-in feature for this type of normalization in pytorch

In [5]:
mean_value = trainset.data.mean(axis=(0, 1, 2))
std_value  = trainset.data.std(axis=(0, 1, 2))

# log
print(f"Mean values per channel : {mean_value}")
print(f"STD values per channel  : {std_value}")

Mean values per channel : [125.30691805 122.95039414 113.86538318]
STD values per channel  : [62.99321928 62.08870764 66.70489964]


In [6]:
# standardize with mean:0 and std:1
zscore_trainset_1 = (trainset.data - mean_value) / std_value

# standardize with mean:2 and std:5
zscore_trainset_2 = zscore_trainset_1 * 5 + 2

# log
print(f"Mean values for minmax_trainset_1 : {zscore_trainset_1.mean(axis=(0, 1, 2))}")
print(f"STD values for minmax_trainset_1  : {zscore_trainset_1.std(axis=(0, 1, 2))}")
print('-' * 50)
print(f"Mean values for minmax_trainset_2 : {zscore_trainset_2.mean(axis=(0, 1, 2))}")
print(f"STD values for minmax_trainset_2  : {zscore_trainset_2.std(axis=(0, 1, 2))}")

Mean values for minmax_trainset_1 : [1.90680804e-17 9.16847154e-17 1.50768287e-17]
STD values for minmax_trainset_1  : [1. 1. 1.]
--------------------------------------------------
Mean values for minmax_trainset_2 : [2. 2. 2.]
STD values for minmax_trainset_2  : [5. 5. 5.]


## 2. Train Normalization
   - Batch normalization
   - Layer normalization
   - Instance normalization
   - group normalization

In [7]:
transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
    ]
)

trainset = CIFAR10('./dataset', train=True, transform=transform)

In [8]:
batch_size = 8
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=False)

first_batch = next(iter(trainloader))[0]

# log
print(f"first_batch.shape : {first_batch.shape}")
print(f"first_batch.dtype : {first_batch.dtype}")
print(f"type.first_batch) : {type(first_batch)}")

first_batch.shape : torch.Size([8, 3, 32, 32])
first_batch.dtype : torch.float32
type.first_batch) : <class 'torch.Tensor'>


In [9]:
in_channels = first_batch.shape[1]
out_channels = 16

model = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3)
)

features_maps = model(first_batch)

# log
print(f"features_maps.shape : {features_maps.shape}")

features_maps.shape : torch.Size([8, 16, 30, 30])


### 2.1. Batch Normalization

In [10]:
bn_mean = features_maps.mean(dim=(0, 2, 3), keepdim=True)
bn_std  = features_maps.std(dim=(0, 2, 3), keepdim=True)

bn_result_1 = (features_maps - bn_mean) / bn_std
bn_result_2 = torch.nn.BatchNorm2d(out_channels, affine=False, eps=0)(features_maps)

# log
print(f"bn_mean.shape : {bn_mean.shape}")
print(f"bn_std.shape  : {bn_std.shape}")
print(torch.allclose(bn_result_1, bn_result_2, atol=1e-03))

bn_mean.shape : torch.Size([1, 16, 1, 1])
bn_std.shape  : torch.Size([1, 16, 1, 1])
True


### 2.2. Layer Normalization

In [11]:
ln_mean = features_maps.mean(dim=(1, 2, 3), keepdim=True)
ln_std  = features_maps.std(dim=(1, 2, 3), keepdim=True)

ln_result_1 = (features_maps - ln_mean) / ln_std
ln_result_2 = torch.nn.LayerNorm(features_maps.shape[1:], elementwise_affine=False, eps=0)(features_maps)

# log
print(f"ln_mean.shape : {ln_mean.shape}")
print(f"ln_std.shape  : {ln_std.shape}")
print(torch.allclose(ln_result_1, ln_result_2, atol=1e-03))

ln_mean.shape : torch.Size([8, 1, 1, 1])
ln_std.shape  : torch.Size([8, 1, 1, 1])
True


### 2.3. Instance Normalization

In [12]:
in_mean = features_maps.mean(dim=(2, 3), keepdim=True)
in_std  = features_maps.std(dim=(2, 3), keepdim=True)

in_result_1 = (features_maps - in_mean) / in_std
in_result_2 = torch.nn.InstanceNorm2d(out_channels, affine=False, eps=0)(features_maps)

# log
print(f"in_mean.shape : {in_mean.shape}")
print(f"in_std.shape  : {in_std.shape}")
print(torch.allclose(in_result_1, in_result_2, atol=1e-02))

in_mean.shape : torch.Size([8, 16, 1, 1])
in_std.shape  : torch.Size([8, 16, 1, 1])
True


### 2.4. Group Normalization

In [13]:
groups = [features_maps[:, :8, :, :], features_maps[:, 8:, :, :]]

gn_mean_1 = groups[0].mean(dim=(1, 2, 3), keepdim=True)
gn_std_1  = groups[0].std(dim=(1, 2, 3), keepdim=True)
result_1  = (groups[0] - gn_mean_1) / gn_std_1

gn_mean_2 = groups[1].mean(dim=(1, 2, 3), keepdim=True)
gn_std_2  = groups[1].std(dim=(1, 2, 3), keepdim=True)
result_2  = (groups[1] - gn_mean_2) / gn_std_2

gn_result_1 = torch.concatenate([result_1, result_2], dim=1)
gn_result_2 = torch.nn.GroupNorm(num_groups=2, num_channels=out_channels, affine=False)(features_maps)

# log
print(f"gn_mean_1.shape : {gn_mean_1.shape}")
print(f"gn_std_1.shape  : {gn_std_1.shape}")
print(f"gn_mean_2.shape : {gn_mean_2.shape}")
print(f"gn_std_2.shape  : {gn_std_2.shape}")
print(torch.allclose(gn_result_1, gn_result_2, atol=1e-03))

gn_mean_1.shape : torch.Size([8, 1, 1, 1])
gn_std_1.shape  : torch.Size([8, 1, 1, 1])
gn_mean_2.shape : torch.Size([8, 1, 1, 1])
gn_std_2.shape  : torch.Size([8, 1, 1, 1])
True
